In [1]:
! pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 199 kB 61.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=0af5aef1c3f8e1d5ece67decf4b5ea28e1c69f20d49ae29e5c80d5304efa6900
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
# Import Spark Session
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('spark').getOrCreate()

In [4]:
from google.colab import files
files.upload()

Saving Admission_Predict_Ver1.1.csv to Admission_Predict_Ver1.1.csv


{'Admission_Predict_Ver1.1.csv': b'Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR ,CGPA,Research,Chance of Admit \r\n1,337,118,4,4.5,4.5,9.65,1,0.92\r\n2,324,107,4,4,4.5,8.87,1,0.76\r\n3,316,104,3,3,3.5,8,1,0.72\r\n4,322,110,3,3.5,2.5,8.67,1,0.8\r\n5,314,103,2,2,3,8.21,0,0.65\r\n6,330,115,5,4.5,3,9.34,1,0.9\r\n7,321,109,3,3,4,8.2,1,0.75\r\n8,308,101,2,3,4,7.9,0,0.68\r\n9,302,102,1,2,1.5,8,0,0.5\r\n10,323,108,3,3.5,3,8.6,0,0.45\r\n11,325,106,3,3.5,4,8.4,1,0.52\r\n12,327,111,4,4,4.5,9,1,0.84\r\n13,328,112,4,4,4.5,9.1,1,0.78\r\n14,307,109,3,4,3,8,1,0.62\r\n15,311,104,3,3.5,2,8.2,1,0.61\r\n16,314,105,3,3.5,2.5,8.3,0,0.54\r\n17,317,107,3,4,3,8.7,0,0.66\r\n18,319,106,3,4,3,8,1,0.65\r\n19,318,110,3,4,3,8.8,0,0.63\r\n20,303,102,3,3.5,3,8.5,0,0.62\r\n21,312,107,3,3,2,7.9,1,0.64\r\n22,325,114,4,3,2,8.4,0,0.7\r\n23,328,116,5,5,5,9.5,1,0.94\r\n24,334,119,5,5,4.5,9.7,1,0.95\r\n25,336,119,5,4,3.5,9.8,1,0.97\r\n26,340,120,5,4.5,4.5,9.6,1,0.94\r\n27,322,109,5,4.5,3.5,8.8,0,0.76\r\n28,298,9

In [8]:
df = spark.read.csv('Admission_Predict_Ver1.1.csv',inferSchema=True, header =True)
df.show(5)

+----------+---------+-----------+-----------------+---+----+----+--------+----------------+
|Serial No.|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+----------+---------+-----------+-----------------+---+----+----+--------+----------------+
|         1|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|
|         2|      324|        107|                4|4.0| 4.5|8.87|       1|            0.76|
|         3|      316|        104|                3|3.0| 3.5| 8.0|       1|            0.72|
|         4|      322|        110|                3|3.5| 2.5|8.67|       1|             0.8|
|         5|      314|        103|                2|2.0| 3.0|8.21|       0|            0.65|
+----------+---------+-----------+-----------------+---+----+----+--------+----------------+
only showing top 5 rows



In [6]:
print((df.count(), len(df.columns)))
df.columns # Tha data has 500 rows, and 9 columns

(500, 9)


['Serial No.',
 'GRE Score',
 'TOEFL Score',
 'University Rating',
 'SOP',
 'LOR ',
 'CGPA',
 'Research',
 'Chance of Admit ']

In [7]:
# Print schema
df.printSchema()

root
 |-- Serial No.: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOEFL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR : double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit : double (nullable = true)



In [9]:
# Get the summary statistics
df.describe().show()

+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|summary|       Serial No.|         GRE Score|      TOEFL Score|University Rating|               SOP|              LOR |              CGPA|          Research|   Chance of Admit |
+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|  count|              500|               500|              500|              500|               500|               500|               500|               500|                500|
|   mean|            250.5|           316.472|          107.192|            3.114|             3.374|             3.484| 8.576440000000003|              0.56| 0.7217399999999996|
| stddev|144.4818327679989|11.295148372354712|6.081867659564538|1.143511800759815|0.9910036207566072|0.92

In [10]:
# Drop the nessesary columns
df = df.drop('Serial No.')
df.show()

+---------+-----------+-----------------+---+----+----+--------+----------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+---------+-----------+-----------------+---+----+----+--------+----------------+
|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|
|      324|        107|                4|4.0| 4.5|8.87|       1|            0.76|
|      316|        104|                3|3.0| 3.5| 8.0|       1|            0.72|
|      322|        110|                3|3.5| 2.5|8.67|       1|             0.8|
|      314|        103|                2|2.0| 3.0|8.21|       0|            0.65|
|      330|        115|                5|4.5| 3.0|9.34|       1|             0.9|
|      321|        109|                3|3.0| 4.0| 8.2|       1|            0.75|
|      308|        101|                2|3.0| 4.0| 7.9|       0|            0.68|
|      302|        102|                1|2.0| 1.5| 8.0|       0|             0.5|
|      323|     

In [11]:
# Check the Null values
for i in df.columns:
    print(i+':',df[df[i].isNull()].count())

GRE Score: 0
TOEFL Score: 0
University Rating: 0
SOP: 0
LOR : 0
CGPA: 0
Research: 0
Chance of Admit : 0


In [12]:
# Correlation analysis
for col in df.columns:
    print('Corellation to chance of admit col for {} is {}'.format(
          col, df.stat.corr('Chance of Admit ',col)))

Corellation to chance of admit col for GRE Score is 0.8103506354632598
Corellation to chance of admit col for TOEFL Score is 0.7922276143050823
Corellation to chance of admit col for University Rating is 0.6901323687886892
Corellation to chance of admit col for SOP is 0.6841365241316723
Corellation to chance of admit col for LOR  is 0.6453645135280112
Corellation to chance of admit col for CGPA is 0.882412574904574
Corellation to chance of admit col for Research is 0.5458710294711379
Corellation to chance of admit col for Chance of Admit  is 1.0


In [13]:
# Feature selection
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['GRE Score','TOEFL Score','CGPA'],
                            outputCol='features')

# display dataframe
output_data = assembler.transform(df)
output_data.show()


+---------+-----------+-----------------+---+----+----+--------+----------------+------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |          features|
+---------+-----------+-----------------+---+----+----+--------+----------------+------------------+
|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|[337.0,118.0,9.65]|
|      324|        107|                4|4.0| 4.5|8.87|       1|            0.76|[324.0,107.0,8.87]|
|      316|        104|                3|3.0| 3.5| 8.0|       1|            0.72| [316.0,104.0,8.0]|
|      322|        110|                3|3.5| 2.5|8.67|       1|             0.8|[322.0,110.0,8.67]|
|      314|        103|                2|2.0| 3.0|8.21|       0|            0.65|[314.0,103.0,8.21]|
|      330|        115|                5|4.5| 3.0|9.34|       1|             0.9|[330.0,115.0,9.34]|
|      321|        109|                3|3.0| 4.0| 8.2|       1|            0.75| [321.0,10

In [14]:
# Import LinearRegression, and create final data
from pyspark.ml.regression import LinearRegression

final_data = output_data.select('features', 'Chance of Admit ')

# Print schema of final data
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Chance of Admit : double (nullable = true)



In [15]:
# Split the data into training and testing set
train, test = final_data.randomSplit([0.7, 0.3])

In [20]:
# Build and train the model
models = LinearRegression(featuresCol='features',labelCol='Chance of Admit ')
model = models.fit(train)

# Get summary of the model
summary = model.summary

# Print rmse,r2,MSE,MAE
print('\033[1m' + 'RMSE:',summary.rootMeanSquaredError)
print('R2 score:', summary.r2)
print("MSE = %s" % summary.meanSquaredError)
print("MAE = %s" % summary.meanAbsoluteError)


RMSE: 0.059485381235535005
R2 score: 0.8088042440518557
MSE = 0.00353851058073694
MAE = 0.04269712376859878


In [17]:
# Transform on the test data
predictions = model.transform(test)

# Display the predictions
predictions.show(20)

+------------------+----------------+-------------------+
|          features|Chance of Admit |         prediction|
+------------------+----------------+-------------------+
| [294.0,93.0,7.36]|            0.46| 0.4528988364793167|
| [295.0,96.0,7.34]|            0.47| 0.4616363084447421|
| [295.0,99.0,7.65]|            0.57| 0.5158282472783413|
|  [296.0,97.0,7.8]|            0.49| 0.5335841773362446|
| [296.0,99.0,7.28]|            0.47|0.46458708005200444|
|[296.0,101.0,7.68]|             0.6| 0.5286840910055277|
| [297.0,96.0,7.43]|            0.34|  0.479228006423964|
| [297.0,99.0,7.81]|            0.54| 0.5435466708843495|
|[297.0,101.0,7.67]|            0.57| 0.5295232270026635|
| [298.0,97.0,7.21]|            0.45|0.45280196922668425|
|  [298.0,98.0,7.5]|            0.44| 0.4978705505093144|
| [298.0,98.0,8.03]|            0.34| 0.5745443302549824|
|  [298.0,99.0,7.6]|            0.46| 0.5154523050906668|
|[298.0,100.0,7.95]|            0.58| 0.5692009369105424|
|[298.0,105.0,

In [18]:
# Evaluate the model
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol='prediction',
                                labelCol='Chance of Admit ',
                                metricName='r2')

print('\033[1m' + 'R2 on the test data:',evaluator.evaluate(predictions)) 


R2 on the test data: 0.7893714357475847
